# Model Evaluation

This notebook evaluates the performance of the trained model on the test dataset. It calculates metrics such as accuracy, precision, recall, and F1-score.

In [ ]:
# Import necessary libraries
import numpy as np
import sys
sys.path.append(os.path.abspath('../src'))
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from sklearn.metrics import classification_report, confusion_matrix
from model import create_model

# Load the trained model
model = create_model(num_classes=3)
model.load_state_dict(torch.load('best_model.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Prepare the test data generator
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(root='../data/processed/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluate the model
true_classes = []
predicted_classes = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_classes.extend(labels.cpu().numpy())
        predicted_classes.extend(preds.cpu().numpy())

# Generate classification report
class_labels = list(test_dataset.class_to_idx.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

ModuleNotFoundError: No module named 'model'